In [2]:
# This is a covpy of New_Data that tries to localize data normalization to the training loop
datadir = "../data/"
datafilename_network = "Link_info_network_and_proj.xlsx"
datafilename_proj = 'Certainty.xlsx'
datafilepath_network = datadir + datafilename_network
datafilepath_proj = datadir + datafilename_proj

from itertools import combinations
import pandas as pd
import numpy as np
import math
import torch
from torch.utils.data import Subset
import wandb


# # start a new wandb run to track this script
# wandb.init(
#     # set the wandb project where this run will be logged
#     project="CGN to predict AADT",

#     # track hyperparameters and run metadata
#     config={
#      "hidden_layer_sizes": [12,24,48,64],
#     }
# )

network_cols_used = ['LINK','ANODE','BNODE','LENGTH','A X_COORD','A Y_COORD','B X_COORD','B Y_COORD','LANES_AB','LEFT_AB', 'RIGHT_AB','SPEED_AB','FSPD_AB','CAP_AB','LANES_BA','LEFT_BA','RIGHT_BA','SPEED_BA','FSPD_BA','CAP_BA'
]

# all 30000 links in the general network
df_network= pd.read_excel(datafilepath_network,sheet_name='Link_Info',usecols=network_cols_used).dropna(subset=['LINK'])

#650 links in the 6 projects
df_proj= pd.read_excel(datafilepath_network, sheet_name='Project_Links')


import networkx as nx

# Create a directed graph for the entire network
G_network = nx.DiGraph()

Anodes = df_network['ANODE'].tolist
Bnodes = df_network['BNODE'].tolist

# Add edges_ab with attributes
edges = [
    (row['ANODE'], row['BNODE'], {
        'length': row['LENGTH'], 
        '#lanes': row['LANES_AB'], 
        'speed': row['SPEED_AB'], 
        'FSPD': row['FSPD_AB'],
        'capacity': row['CAP_AB'],
        'Link ID': row['LINK'],
        'AADT Before': 0,
        'auto volume before': 0,
        'VMT before': 0
    })
    for _, row in df_network.iterrows()
]

# Add edges_ba only if lanes_ba != 0
edges += [
    (row['BNODE'], row['ANODE'], {
        'length': row['LENGTH'], 
        '#lanes': row['LANES_BA'], 
        'speed': row['SPEED_BA'], 
        'FSPD': row['FSPD_BA'],
        'capacity': row['CAP_BA'],
        'Link ID': row['LINK'],
        'AADT Before': 0,
        'auto volume before': 0,
        'VMT before': 0
    })
    for _, row in df_network.iterrows() if row['LANES_BA'] != 0
]

# List of pairs of integers that corresond to eges in project
proj_links = list(zip(df_proj['ANODE'], df_proj['BNODE']))


# Identifying edges in G_network that correspond to the list of pairs
# proj_edges = [(u, v) for u, v in proj_links if G_network.has_edge(u, v)]

# for project links, the "AADT Before","auto voulme before" and "VMT before" attributes are known and not zero
for i, item in enumerate(edges):
    if (edges[i][0], edges[i][1]) in proj_links:
        idx = proj_links.index((edges[i][0], edges[i][1]))
        edges[i][2]['AADT Before'] = df_proj.loc[idx]['AADT(2010)-B']
        edges[i][2]['auto volume before'] = df_proj.loc[idx]['auto volume(2010)-B']
        edges[i][2]['VMT before'] = df_proj.loc[idx]['VMT-B']
        
    


G_network.add_edges_from(edges)


# Creating the subgraph with only project edges
G_proj = G_network.edge_subgraph(proj_links)


G_network_dual = nx.line_graph(G_network) # dual graph of G_network
G_proj_dual = nx.line_graph(G_proj) # dual graph of G_proj


node_attrs = {}
for edge in G_network_dual.nodes:
    u, v = edge
    edge_data = G_network[u][v]  # Get the attributes of the edge from the original graph
    node_attrs[edge] = f"length: {edge_data['length']}, #lanes: {edge_data['#lanes']}, speed: {edge_data['speed']}, FSPD: {edge_data['FSPD']}, capacity: {edge_data['capacity']}, Link ID: {edge_data['Link ID']}, AADT Before: {edge_data['AADT Before']}, auto volume before: {edge_data['auto volume before']}, VMT before: {edge_data['VMT before']}"

# Convert dictionary to DataFrame
def parse_attributes(attr_string):
    attributes = {}
    for item in attr_string.split(', '):
        key, value = item.split(': ')
        attributes[key] = float(value)
    return attributes

# Create a DataFrame
df_node_attr_network = pd.DataFrame(
    [dict(**parse_attributes(value), index=key) for key, value in node_attrs.items()],
    index=[key for key in node_attrs.keys()]
)

# the dataframe that contains all node attributes in the dual network graph
df_node_attr_network = df_node_attr_network.iloc[:, :-1]

df_network_link_id = df_node_attr_network['Link ID']

# reorder columns of the data frame so that Link ID is the last column
df_node_attr_network = df_node_attr_network.iloc[:,[0,1,2,3,4,6,7,8,5]]

# Mapping node labels in G_network_dual, which are pairs of integers that coorespond to node labels in G_network,
# to the natural numbers 
node_mapping = {node: i + 1 for i, node in enumerate(G_network_dual.nodes())}
# Step 3: Construct a new graph with edges represented by mapped values
mapped_G_network_dual = nx.DiGraph()

# Add nodes with mapped attributes
for original_node in G_network_dual.nodes(data=True):
    original_label, attributes = original_node
    new_label = node_mapping[original_label]
    
    # Add the new node with the same attributes
    mapped_G_network_dual.add_node(new_label, **attributes)


# Add edges to the mapped graph using the new labels
for u, v in G_network_dual.edges():
    new_u = node_mapping[u]
    new_v = node_mapping[v]
    mapped_G_network_dual.add_edge(new_u, new_v)

# Identifying vertices in G_network_dual that correspond to links in G_proj
proj_nodes = [pair for pair in proj_links if pair in G_network_dual.nodes()]

# Keep the first occurrence of each unique element in proj_nodes
# proj_nodes_unique = list(dict.fromkeys(proj_nodes))





In [8]:

from torch_geometric.nn import GCNConv
import torch.nn as nn
import torch.nn.functional as F

node_attr_used = df_node_attr_network.columns[:-1]
num_attr = len(node_attr_used)

num_node_label = 3

class GCN(torch.nn.Module):
    def __init__(self, hidden_channels):
        super().__init__()
        self.conv1 = GCNConv(num_attr, hidden_channels)
        self.conv2 = GCNConv(hidden_channels,hidden_channels)
        self.conv3 = GCNConv(hidden_channels, num_attr)
        self.head = nn.Linear(num_attr, num_node_label)
        # add a multi-layer perceptron?
        # reference examples of using GNN for classification eg. citation network
        # can pretty much reuse all the previous layers (backbone) except for the last layer (layer head) which is specific to the application 



    def forward(self, x, edge_index):
        x = self.conv1(x, edge_index)
        x = x.relu()

        x = self.conv2(x, edge_index)
        x = x.relu()

        x = self.conv3(x, edge_index)
        x = x.relu()
        
        x = self.head(x)
        
        # x = x.relu() # no activation function
        # x = F.dropout(x, p=0.5, training=self.training)
        # x = self.conv2(x, edge_index)
        return x


###### Try this example


In [3]:
# create a list of all sheet names

num_projs = 6
proj_list = list(range(1,num_projs+1))
def generate_combinations(proj_list):
    all_combinations = []
    # Loop through lengths from 1 to 6
    for length in range(1, 1+num_projs):
        # Generate combinations of the current length
        comb = combinations(proj_list, length)
        # Convert each combination to a string, add "p", and add to the list
        all_combinations.extend(['P' + ''.join(map(str, c)) for c in comb])
    return all_combinations
# Generate all combinations
sheet_names = generate_combinations(proj_list)

# Insert "P0" at the beginning of the list
# sheet_names.insert(0, "P0")


In [4]:
# import torch
from torch_geometric.data import Data
from torch_geometric.loader import DataLoader
# Prepare the dataset 

proj_cols_used = ['Link ID','ANODE','BNODE','A X_COORD','A Y_COORD','B X_COORD','B Y_COORD','Link Length(miles)','# of lanes-A','Capacity-A (veh/h)',
            'auto volume(2010)-A','AADT(2010)-B','AADT(2010)-A','Speed(mph)-A','VMT-A']
proj_features_used = ['']
dataset = []

# edge index is shared among all samples
# Get the list of edges as tuples (source, target)
# the entire network has edge_index 
edges = list(mapped_G_network_dual.edges())

# Separate source and target nodes
source_nodes = [edge[0] for edge in edges]
target_nodes = [edge[1] for edge in edges]

# Create the edge_index tensor: shape [2, num_edges]
edge_index = torch.tensor([source_nodes, target_nodes], dtype=torch.long)-1


for ii in range(len(sheet_names)):
    # read the correct sheet 
    df_sample= pd.read_excel(datafilepath_proj,sheet_name=sheet_names[ii],usecols=proj_cols_used).dropna(subset=['Link ID'])
    df_sample = df_sample.astype('float64')
    df_sample_label = df_sample[['AADT(2010)-A','auto volume(2010)-A','VMT-A']]
    df_sample_label.columns = ['AADT after','auto volume after','VMT after']
    # # scale AADT values in df_sample 
    # df_sample_scaled = scaler.fit_transform(pd.DataFrame(df_sample[['AADT(2010)-A','auto volume(2010)-A','VMT-A']]))
    # # put the scaled data into a dataframe
    # df_sample_scaled = pd.DataFrame(df_sample_scaled,index=df_sample.index, columns=['scaled AADT after','scaled auto volume after','scaled VMT after'])


    # x = node features
    # The entire network has node features
    df_node_attr_network_used = df_node_attr_network.iloc[:, :-1]
    x = torch.tensor(np.array(df_node_attr_network_used), dtype=torch.float)
################################################################################   
    # # edge index
    # already defined as above
###################################################################################
    
    # Create the label tensor y
        
    # -1 for unlabeled nodes, correct class labels (positive real value) for labeled nodes in G_proj_dual
    # create a all -1 dataframe to store labels of the entire network
    df_labels_network = pd.DataFrame(-2, index = df_node_attr_network.index, columns = ['Link ID','AADT after','auto volume after','VMT after'])
    network_links = list(G_network_dual.nodes)

    df_labels_network['Link ID'] = df_network_link_id

    # for index in range(1, len(df_sample)+1):  # Loop through each row in df_sample
    #     link_id = df_sample.loc[index, 'Link ID']  # Get 'Link ID' for the current row
    
    # # Find the row in df_scaled_labels_network where 'Link ID' matches
    #     row_idx = df_scaled_labels_network[df_scaled_labels_network['Link ID'] == link_id].index
    
    #     if not row_idx.empty:  # If a match is found
    #     # Assign the value of 'AADT(2010)-A' from df_sample to 'AADT_after' in df_scaled_labels_network
    #         df_scaled_labels_network = df_scaled_labels_network.astype('float64')
    #         df_scaled_labels_network.loc[row_idx, 'scaled AADT after'] = df_sample_scaled.loc[index, 'scaled AADT after']
    #         df_scaled_labels_network.loc[row_idx, 'scaled auto volume after'] = df_sample_scaled.loc[index, 'scaled auto volume after']
    #         df_scaled_labels_network.loc[row_idx, 'scaled VMT after'] = df_sample_scaled.loc[index, 'scaled VMT after']

    for index in range(1,len(df_sample)+1):  # Loop through each row in df_sample
        pairwise_link_id = list(zip(df_sample.loc[:,'ANODE'], df_sample.loc[:,'BNODE']))[index-1]  # Get 'pairwise link ID' for the current row
    
    # Find the row in df_scaled_labels_network where 'pairwise Link ID' matches
        row_idx = df_labels_network[df_labels_network.index == pairwise_link_id].index
    
        if not row_idx.empty:  # If a match is found
        # Assign the value of 'AADT(2010)-A' from df_sample to 'AADT_after' in df_scaled_labels_network
          
            df_labels_network = df_labels_network.astype('float64')
            df_labels_network.loc[row_idx, 'AADT after'] = df_sample_label.loc[index, 'AADT after']
            df_labels_network.loc[row_idx, 'auto volume after'] = df_sample_label.loc[index, 'auto volume after']
            df_labels_network.loc[row_idx, 'VMT after'] = df_sample_label.loc[index, 'VMT after']

    
    y = torch.tensor(np.array(df_labels_network[['AADT after','auto volume after','VMT after']]), dtype=torch.float)  

##################################################################################  
    # Create the train_mask so that training only happens on project links
    train_mask = pd.DataFrame(False, index = df_node_attr_network.index, columns = ['Masked'])

    row_idx = df_labels_network[df_labels_network['AADT after'] != -2].index

    train_mask.loc[row_idx] = True
    train_mask = torch.tensor(train_mask.values, dtype=torch.bool)

    new_data = Data(x=x, edge_index=edge_index, y=y, train_mask=train_mask)


    # Append all 63 samples
    dataset.append(new_data)


/home/duantu/soft/transportation_venv/lib/python3.12/site-packages/torch/cuda/__init__.py:619: UserWarning: Can't initialize NVML
  warnings.warn("Can't initialize NVML")


In [5]:
# define our data loader
from torch.utils.data import Dataset

class CustomDataset(Dataset):
    
    def __init__(self,data):
        super().__init__()
        self.data = data

    def __len__(self):
        return len(self.data)

    def __getitem__(self, index):
        x = self.data[index]['x']
        edge_index = self.data[index]['edge_index']
        target = self.data[index]['y']
        return x, edge_index, target


In [6]:

val_indices =[26, 19, 52, 11, 50, 40, 46, 39, 60, 29, 21, 12, 24, 48, 10, 17, 32, 13]

val_dataset = Subset(dataset, val_indices)

val_dataset = CustomDataset(val_dataset)

val_loader = DataLoader(val_dataset, batch_size=1, shuffle=False)

In [9]:
# Training settings
# model = GCN(hidden_channels=24)
# state = torch.load( "./checkpoint.pth.tar")
# model.load_state_dict(state["model_state"])
# criterion = torch.nn.MSELoss()



model = GCN(hidden_channels=24)
state = torch.load( "./savedModels/GCN model epoch_1143.pth")
model.load_state_dict(torch.load('./savedModels/GCN model epoch_1143.pth', weights_only=True))
criterion = torch.nn.MSELoss()


In [ ]:
# SCALE THINGS LAST MINUTE 

from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler

# scaler = MinMaxScaler()
scaler = StandardScaler()

sum_target_2_norm_squared_AADT = 0
for i, (x, edge_index, target) in enumerate(val_loader):
    target_scaled = torch.tensor(scaler.fit_transform(target[0]))
    sum_target_2_norm_squared_AADT += criterion(target_scaled[train_mask.squeeze()][:,0], torch.zeros(target_scaled[train_mask.squeeze()][:,0].shape))
normalizing_const_ave_val_AADT = sum_target_2_norm_squared_AADT/len(val_loader)


total_val_loss_AADT = 0
df_rel_err = pd.DataFrame(columns = val_indices, index = np.array(df_node_attr_network['Link ID'],dtype = int).reshape(-1,1)[np.array(train_mask)==1])

# df_unscaled_AADT_comparison = pd.DataFrame(columns = [sheet_names[i] for i in df_rel_err.columns],index = proj_order_list)

for i, (x, edge_index, target) in enumerate(val_loader):
    x_scaled = torch.tensor(scaler.fit_transform(x[0]))
    x_scaled = x_scaled.to(torch.float32)
    target_scaled = torch.tensor(scaler.fit_transform(target[0]))
    out = model(x_scaled, edge_index[0]).squeeze()
    normalizing_constant_single_val_set_AADT = criterion(target_scaled[train_mask.squeeze()][:,0], torch.zeros(target_scaled[train_mask.squeeze()][:,0].shape))
    
    loss_AADT = criterion(out[train_mask.squeeze()][:,0], target_scaled[train_mask.squeeze()][:,0])
    # print(out[train_mask.squeeze()])
    # print(target[0][train_mask.squeeze()][:,0])
    relative_loss_AADT = loss_AADT/normalizing_constant_single_val_set_AADT.item()
    
    print(f'\nVal sample # {val_indices[i]} Absolute MSE (AADT): {format(loss_AADT,".2e")}') 
    print(f'Val sample # {val_indices[i]} Relative MSE (AADT): {format(relative_loss_AADT,".2e")}')

    #########
    unscaled_output = scaler.inverse_transform(out[train_mask.squeeze()].detach().numpy())[:,0]
    unscaled_node_label = scaler.inverse_transform(target[0][train_mask.squeeze()].detach().numpy())[:,0] 
 

    # # convert unscaled_label into a data frame
    # df_unscaled_node_label = pd.DataFrame(data = unscaled_node_label)
    # df_unscaled_node_label.columns = ['unscaled_node_label']
    # df_unscaled_node_label.index = df_node_attr_network.index[train_mask.squeeze()]
    # sorted_df_unscaled_node_label = df_unscaled_node_label.reindex(sorted_output_indices)

    # df_real_label = pd.read_excel(datafilepath_proj,sheet_name=sheet_names[val_indices[i]],usecols=proj_cols_used).dropna(subset=['Link ID'])['AADT(2010)-A']

    # print(list(zip(sorted_df_unscaled_node_label['unscaled_node_label'], df_real_label)))
    

    
    df_rel_err.iloc[:,i] = abs(unscaled_output-unscaled_node_label)/unscaled_node_label

    num = 0
    for j in range(len(unscaled_node_label)):
        if abs(unscaled_output[j]-unscaled_node_label[j])/unscaled_node_label[j]>0.9:
            num+=1
    # print([abs(unscaled_output-unscaled_node_label)/unscaled_node_label][0][491])
    print('validation sample #', val_indices[i], 'has ',num, 'links with relative error > 0.9')
    ########
    
    total_val_loss_AADT += loss_AADT.item()
      
# normalizing constant: squared l2 norm divided by length of y[train_mask] 
relative_ave_val_mse_AADT = total_val_loss_AADT/len(val_loader)/normalizing_const_ave_val_AADT.item()
print(f"\nRelative average validation set mse (AADT) = {total_val_loss_AADT/len(val_loader):.2e} / {normalizing_const_ave_val_AADT.item():.2e} = {relative_ave_val_mse_AADT:.2e}")

# make a copy of the data frame that has all the relative error info of each validation sample 
import copy
df_rel_err_copy = copy.copy(df_rel_err)


Val sample # 26 Absolute MSE (AADT): 9.16e+01
Val sample # 26 Relative MSE (AADT): 9.34e-01
